<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=bd6f60e80e6f0df9877b7217089b88c960dfa89db001c858a5bfa21eb22fde4e
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-08 10:41:23
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.24 C
CY Investment:  1.51 C
Reserve:  57.44 K
Current:  1.41 C
-------------------
Today PnL: -16.03 K (-0.11%)
Current PnL: -20.77 L (-13.78%)
CY Booked + Current PnL: -9.33 L (-6.19%)
-------------------
Total profit:  1.33 L
Total loss:  -22.10 L
-------------------
Total Booked + Current PnL: 17.65 L (14.27%)
Total Booked PnL: 38.42 L (31.06%)
Curr Year Booked PnL: 11.44 L (8.12%)
Prev Year Booked PnL: 26.98 L (21.81%)
Est FTT:  2.29 C
Est FTT PnL: 88.28 L (62.69%)
Deployed:  1.24 C
Current:  1.41 C
CAGR/XIRR %: 7.85%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-40.35,68.0,H-MC,14.17,118407.0,17903.0,11201.0,2.92,17.81,9.46,28.96,79.0,1.60,0.84,57.83,MH,ATH,METALS
77,TTKPRESTIG,770.00,93.38,39.0,M-SC,1.18,83323.0,-17454.0,17548.0,-0.05,-17.32,21.06,0.09,245.0,-0.99,0.59,9.47,OX40N,NTT,DURABLES
14,BLUESTARCO,2080.00,9.17,46.0,H-MC,3.02,188390.0,23720.0,19611.0,-1.48,14.40,10.41,26.31,89.0,1.21,1.34,23.54,X40N,NTT,AC
19,CIPLA,1662.74,-22.87,40.0,H-LC,3.81,205801.0,1301.0,22000.0,-0.73,0.64,10.69,11.39,10.0,0.06,1.47,9.64,X40N,ATH,PHARMA
57,RELIANCE,1533.00,-14.04,48.0,H-LC,3.23,214906.0,4540.0,24241.0,-0.52,2.16,11.28,13.68,37.0,0.19,1.53,19.04,XY25,NTT,REFINERIES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TITAGARH,1548.00,-1.01,60.0,H-SC,4.84,211413.0,-30764.0,141541.0,3.16,-12.70,66.95,45.74,158.0,-0.22,1.51,37.50,XY24,NTT,ENGINEERING
50,NATIONALUM,244.55,-40.35,68.0,H-MC,14.17,118407.0,17903.0,11201.0,2.92,17.81,9.46,28.96,79.0,1.60,0.84,57.83,MH,ATH,METALS
33,HONAUT,58357.33,-21.77,51.0,H-SC,2.13,110220.0,-17718.0,64853.0,2.87,-13.85,58.84,36.84,143.0,-0.27,0.79,12.84,X40N,ATH,ELECTRICAL
40,INFY,2275.00,-18.52,52.0,H-LC,8.01,317114.0,4226.0,167468.0,2.08,1.35,52.81,54.87,3.0,0.03,2.26,8.09,X40,BTT,IT
74,TCS,4406.34,-28.69,51.0,H-LC,12.17,284519.0,-61437.0,129684.0,1.79,-17.76,45.58,19.73,1.0,-0.47,2.03,4.79,X40,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,RAJOOENG,143.10,-42.01,39.0,H-SC,15.96,87780.0,-14720.0,55319.0,-1.93,-14.36,63.02,39.61,114.0,-0.27,0.63,7.21,AR,ATH,MISC
46,KPIGREEN,731.05,-10.17,27.0,H-SC,6.55,107100.0,-18197.0,77123.0,-1.62,-14.52,72.01,47.03,141.0,-0.24,0.76,34.10,MH,ATH,POWER
14,BLUESTARCO,2080.00,9.17,46.0,H-MC,3.02,188390.0,23720.0,19611.0,-1.48,14.40,10.41,26.31,89.0,1.21,1.34,23.54,X40N,NTT,AC
10,BANDHANBNK,400.00,-13.31,46.0,H-SC,2.95,217134.0,-61425.0,318861.0,-1.48,-22.05,146.85,92.42,151.0,-0.19,1.55,25.49,XY24,NTT,BANKS
78,UJJIVANSFB,60.00,104.20,59.0,H-SC,13.42,128871.0,-13608.0,33133.0,-1.43,-9.55,25.71,13.70,163.0,-0.41,0.92,52.10,OX40N,NTT,BANKS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.0,43.54,49.0,M-MC,12.45,226133.0,1171.0,166479.0,-1.17,0.52,73.62,74.53,192.0,0.01,1.61,32.94,XY24,BTT,STEEL
35,ICICIPRULI,790.0,-20.88,50.0,H-MC,1.85,136222.0,434.0,42324.0,-0.12,0.32,31.07,31.48,107.0,0.01,0.97,12.48,X40,ATH,INSURANCE
86,WIPRO,420.0,-13.79,48.0,M-LC,5.87,151571.0,626.0,108828.0,0.39,0.41,71.80,72.51,53.0,0.01,1.08,6.46,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,104.20,59.0,H-SC,13.42,128871.0,-13608.0,33133.0,-1.43,-9.55,25.71,13.70,163.0,-0.41,0.92,52.10,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,93.38,39.0,M-SC,1.18,83323.0,-17454.0,17548.0,-0.05,-17.32,21.06,0.09,245.0,-0.99,0.59,9.47,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.79,50.0,H-SC,2.02,222048.0,-47619.0,83956.0,0.10,-17.66,37.81,13.47,138.0,-0.57,1.58,12.90,XY24,NTT,PAINTS
18,CERA,9475.0,-23.39,34.0,H-SC,1.61,139506.0,-36397.0,78416.0,-0.39,-20.69,56.21,23.89,149.0,-0.46,0.99,20.48,OX40N,NTT,CERAMICS
72,TATAELXSI,9161.0,-23.21,48.0,H-MC,6.45,103797.0,-28038.0,70260.0,1.36,-21.27,67.69,32.03,98.0,-0.40,0.74,16.26,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,43.54,49.0,M-MC,12.45,226133.0,1171.0,166479.0,-1.17,0.52,73.62,74.53,192.0,0.01,1.61,32.94,XY24,BTT,STEEL
32,HINDZINC,730.22,31.07,68.0,M-LC,6.90,213911.0,8835.0,103725.0,0.38,4.31,48.49,54.89,52.0,0.09,1.53,29.44,X5K,ATH,METALS
14,BLUESTARCO,2080.00,9.17,46.0,H-MC,3.02,188390.0,23720.0,19611.0,-1.48,14.40,10.41,26.31,89.0,1.21,1.34,23.54,X40N,NTT,AC
84,VOLTAS,1530.00,-2.48,47.0,H-MC,1.28,204750.0,13008.0,24754.0,-0.52,6.78,12.09,19.69,99.0,0.53,1.46,14.04,XY25,NTT,AC
17,CAMS,5211.76,-6.05,46.0,H-SC,1.75,106786.0,4782.0,39148.0,0.35,4.69,36.66,43.06,122.0,0.12,0.76,22.61,X40N,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-40.35,68.0,H-MC,14.17,118407.0,17903.0,11201.0,2.92,17.81,9.46,28.96,79.0,1.60,0.84,57.83,MH,ATH,METALS
11,BANKINDIA,190.00,-25.47,63.0,H-MC,11.14,191496.0,11688.0,100727.0,-1.37,6.50,52.60,62.52,88.0,0.12,1.37,42.80,XR,NTT,BANKS
32,HINDZINC,730.22,31.07,68.0,M-LC,6.90,213911.0,8835.0,103725.0,0.38,4.31,48.49,54.89,52.0,0.09,1.53,29.44,X5K,ATH,METALS
37,INDIAMART,4810.62,-55.39,33.0,H-SC,9.07,124667.0,1331.0,130302.0,0.24,1.08,104.52,106.72,119.0,0.01,0.89,24.71,AR,ATH,MISC
86,WIPRO,420.00,-13.79,48.0,M-LC,5.87,151571.0,626.0,108828.0,0.39,0.41,71.80,72.51,53.0,0.01,1.08,6.46,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-23.05,29.0,H-LC,10.18,277243.0,-38599.0,151264.0,-1.24,-12.22,54.56,35.67,5.0,-0.26,1.98,0.00,X40N,ATH,FMCG
5,ASIANPAINT,3465.66,-18.16,33.0,H-LC,5.72,206686.0,-45082.0,98300.0,-0.17,-17.91,47.56,21.13,27.0,-0.46,1.47,11.18,X40,ATH,PAINTS
31,HINDUNILVR,2922.00,-11.16,38.0,H-LC,7.99,249430.0,-7633.0,42777.0,-0.91,-2.97,17.15,13.67,24.0,-0.18,1.78,16.83,XY25,NTT,FMCG
19,CIPLA,1662.74,-22.87,40.0,H-LC,3.81,205801.0,1301.0,22000.0,-0.73,0.64,10.69,11.39,10.0,0.06,1.47,9.64,X40N,ATH,PHARMA
42,ITC,452.00,-39.61,41.0,H-LC,2.32,195600.0,-4538.0,25428.0,0.05,-2.27,13.00,10.44,4.0,-0.18,1.39,3.17,X40,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.5,0.39,54.0,H-LC,1.96,161220.0,-24875.0,72356.0,-0.81,-13.37,44.88,25.51,15.0,-0.34,1.15,18.81,AR,ATH,ELECTRICAL
42,ITC,452.0,-39.61,41.0,H-LC,2.32,195600.0,-4538.0,25428.0,0.05,-2.27,13.00,10.44,4.0,-0.18,1.39,3.17,X40,NTT,FMCG
48,LTIM,7230.2,-2.89,59.0,H-LC,2.44,240885.0,-5266.0,84478.0,1.59,-2.14,35.07,32.18,16.0,-0.06,1.72,33.13,X200,ATH,IT
51,NESTLEIND,1377.0,-11.53,45.0,H-LC,2.80,272236.0,6810.0,49983.0,-1.14,2.57,18.36,21.40,11.0,0.14,1.94,9.38,XY25,NTT,FMCG
57,RELIANCE,1533.0,-14.04,48.0,H-LC,3.23,214906.0,4540.0,24241.0,-0.52,2.16,11.28,13.68,37.0,0.19,1.53,19.04,XY25,NTT,REFINERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-23.05,29.0,H-LC,10.18,277243.0,-38599.0,151264.0,-1.24,-12.22,54.56,35.67,5.0,-0.26,1.98,0.00,X40N,ATH,FMCG
42,ITC,452.00,-39.61,41.0,H-LC,2.32,195600.0,-4538.0,25428.0,0.05,-2.27,13.00,10.44,4.0,-0.18,1.39,3.17,X40,NTT,FMCG
74,TCS,4406.34,-28.69,51.0,H-LC,12.17,284519.0,-61437.0,129684.0,1.79,-17.76,45.58,19.73,1.0,-0.47,2.03,4.79,X40,ATH,IT
80,UNITDSPR,1644.00,-12.37,51.0,H-LC,6.77,229792.0,-4352.0,52967.0,-1.17,-1.86,23.05,20.77,86.0,-0.08,1.64,4.93,X40N,NTT,BREWERIES
1,ABB,7934.00,-40.16,47.0,H-LC,7.51,249984.0,-11635.0,130842.0,-0.22,-4.45,52.34,45.57,7.0,-0.09,1.78,6.37,AR,NTT,ELECTRICAL


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,0.39,54.0,H-LC,1.96,161220.0,-24875.0,72356.0,-0.81,-13.37,44.88,25.51,15.0,-0.34,1.15,18.81,AR,ATH,ELECTRICAL
42,ITC,452.00,-39.61,41.0,H-LC,2.32,195600.0,-4538.0,25428.0,0.05,-2.27,13.00,10.44,4.0,-0.18,1.39,3.17,X40,NTT,FMCG
19,CIPLA,1662.74,-22.87,40.0,H-LC,3.81,205801.0,1301.0,22000.0,-0.73,0.64,10.69,11.39,10.0,0.06,1.47,9.64,X40N,ATH,PHARMA
5,ASIANPAINT,3465.66,-18.16,33.0,H-LC,5.72,206686.0,-45082.0,98300.0,-0.17,-17.91,47.56,21.13,27.0,-0.46,1.47,11.18,X40,ATH,PAINTS
57,RELIANCE,1533.00,-14.04,48.0,H-LC,3.23,214906.0,4540.0,24241.0,-0.52,2.16,11.28,13.68,37.0,0.19,1.53,19.04,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4406.34,-28.69,51.0,H-LC,12.17,284519.0,-61437.0,129684.0,1.79,-17.76,45.58,19.73,1.0,-0.47,2.03,4.79,X40,ATH,IT
40,INFY,2275.00,-18.52,52.0,H-LC,8.01,317114.0,4226.0,167468.0,2.08,1.35,52.81,54.87,3.0,0.03,2.26,8.09,X40,BTT,IT
42,ITC,452.00,-39.61,41.0,H-LC,2.32,195600.0,-4538.0,25428.0,0.05,-2.27,13.00,10.44,4.0,-0.18,1.39,3.17,X40,NTT,FMCG
83,VBL,671.64,-23.05,29.0,H-LC,10.18,277243.0,-38599.0,151264.0,-1.24,-12.22,54.56,35.67,5.0,-0.26,1.98,0.00,X40N,ATH,FMCG
1,ABB,7934.00,-40.16,47.0,H-LC,7.51,249984.0,-11635.0,130842.0,-0.22,-4.45,52.34,45.57,7.0,-0.09,1.78,6.37,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-45.67,46.0,L-SC,30.87,79872.0,-33677.0,73730.0,0.53,-29.66,92.31,35.27,268.0,-0.46,0.57,95.49,XR,NTT,HOTELS
49,MASFIN,398.61,-19.15,46.0,H-SC,10.89,91950.0,-6030.0,27631.0,-0.89,-6.15,30.05,22.05,152.0,-0.22,0.66,33.66,XR,ATH,FINANCE
50,NATIONALUM,244.55,-40.35,68.0,H-MC,14.17,118407.0,17903.0,11201.0,2.92,17.81,9.46,28.96,79.0,1.60,0.84,57.83,MH,ATH,METALS
78,UJJIVANSFB,60.00,104.20,59.0,H-SC,13.42,128871.0,-13608.0,33133.0,-1.43,-9.55,25.71,13.70,163.0,-0.41,0.92,52.10,OX40N,NTT,BANKS
61,SAMMAANCAP,326.00,-184.09,65.0,M-SC,13.52,144729.0,-4491.0,148666.0,-1.40,-3.01,102.72,96.62,208.0,-0.03,1.03,56.63,XY25,NTT,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAMMAANCAP,326.00,-184.09,65.0,M-SC,13.52,144729.0,-4491.0,148666.0,-1.40,-3.01,102.72,96.62,208.0,-0.03,1.03,56.63,XY25,NTT,FINANCE
69,SURYODAY,240.00,62.28,62.0,H-SC,7.37,148349.0,-30722.0,96457.0,0.12,-17.16,65.02,36.71,135.0,-0.32,1.06,47.19,XR,NTT,BANKS
26,GREENPANEL,537.00,235.27,61.0,M-SC,5.10,149694.0,-33384.0,112360.0,0.25,-18.23,75.06,43.14,230.0,-0.30,1.07,40.81,XY24,NTT,MISC
32,HINDZINC,730.22,31.07,68.0,M-LC,6.90,213911.0,8835.0,103725.0,0.38,4.31,48.49,54.89,52.0,0.09,1.53,29.44,X5K,ATH,METALS
64,SHALBY,327.00,1218.98,63.0,M-SC,21.47,179279.0,-1864.0,47330.0,-0.88,-1.03,26.40,25.10,235.0,-0.04,1.28,43.67,XY24,NTT,HEALTHCARE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.09
1,25,43.35
2,50,75.07


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.15
LC    32.17
MC    23.70
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.07
X40      14.65
X40N     11.17
XR       10.98
XY25     10.33
AR        8.15
OX40N     7.69
X200      1.72
MH        1.60
X5K       1.53
SR        1.13
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.04
H-LC    25.65
H-MC    20.47
M-SC    13.69
M-LC     5.51
M-MC     2.93
L-SC     1.42
L-LC     1.01
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.67,-7.06,42.35
IT,12.78,-17.35,80.91
FINANCE,10.55,-14.06,69.14
MISC,6.92,-18.54,84.22
BANKS,6.31,-12.82,72.17
PAINTS,5.53,-19.03,36.77
ELECTRICAL,5.38,-9.07,47.34
INSURANCE,3.83,-2.97,41.46
AUTO,3.50,-17.51,73.68


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3217083.0,22
XR,1372529.0,14
AR,1188640.0,9
X40,784650.0,10
X40N,669353.0,9
OX40N,568258.0,10
XY25,487975.0,7
SR,262623.0,2
X5K,103725.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3722305.0,29
M-SC,1513340.0,17
H-MC,1282364.0,15
H-LC,1196706.0,15
M-LC,377761.0,4
M-MC,371884.0,2
L-SC,261143.0,3
L-MC,60130.0,1
L-LC,42005.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1194614.0      6
M-SC       XY24       846660.0      7
H-SC       AR         816501.0      5
           XR         800525.0      7
H-MC       XY24       570859.0      4
H-LC       X40        497940.0      5
M-MC       XY24       371884.0      2
H-SC       X40N       307385.0      4
           OX40N      263534.0      4
           SR         262623.0      2
H-LC       X40N       226231.0      3
H-MC       X40        213277.0      4
H-LC       AR         203198.0      2
H-MC       XY25       180303.0      2
L-SC       XR         172205.0      2
M-SC       AR         168941.0      2
M-LC       XY24       165208.0      2
M-SC       XY25       148666.0      1
           OX40N      145526.0      4
H-MC       X40N       135737.0      2
M-SC       XR         130114.0      2
H-LC       XY25       117001.0      3
M-LC       XR         108828.0      1
           X5K        103725.0      1
H-MC       XR         100727.0      1
L-SC       OX40N       88938.0      1
H-LC       X200        84478.0      1
H-SC       MH          77123.0      1
M-SC       X40         73433.0      1
H-MC       OX40N       70260.0      1
H-LC       XY24        67858.0      1
L-MC       XR          60130.0      1
L-LC       XY25        42005.0      1
H-MC       MH          11201.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 26.0 seconds
